In [1]:
#okay so we're going to be looking at how the AnalyzeL2 class within analyze_l2.py works,
#so then we can look at how to create the BCRV Percent variable and then store it within
#def add_headers_L2_barycentric(L2, logger=None): in diagnostics.py
import copy
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
from matplotlib.lines import Line2D
from modules.Utils.kpf_parse import HeaderParse, get_data_products_L2
from astropy.table import Table
from kpfpipe.models.level2 import KPF2

In [2]:
class AnalyzeL2:

    """
    Description:
        This class contains functions to analyze L2 spectra (storing them
        as attributes) and functions to plot the results.

    Arguments:
        L2 - an L2 object

    Attributes:
        name - name of source (e.g., 'Bias', 'Etalon', '185144')
        ObsID - observation  ID (e.g. 'KP.20230704.02326.27')
        header - header of the PRIMARY extension of the L2 object
        rv_header - header of the RV extension
    
    To do:
        Add plot showing combined CCF - https://github.com/Keck-DataReductionPipelines/KPF-Pipeline/issues/940
        Add plot showing correlations between per-order RVs and per-chip RVs and overall RVs.
    """

    def __init__(self, L2, logger=None):
        if logger:
            self.logger = logger
            self.logger.debug('Initializing AnalyzeL2 object')
        else:
            self.logger = None
        self.L2 = copy.deepcopy(L2)
        self.df_RV = self.L2['RV']
        self.n_green_orders = 35
        self.n_red_orders   = 32
        primary_header = HeaderParse(L2, 'PRIMARY')
        self.header = primary_header.header
        self.name = primary_header.get_name()
        if primary_header.get_name(use_star_names=False) in ['Star', 'Sun']:
            self.is_star = True
        else:
            self.is_star = False
        self.ObsID = primary_header.get_obsid()
        self.rv_header = HeaderParse(L2, 'RV').header
        self.df_RVs = self.L2['RV'] # Table of RVs per order and orderlet
        self.data_products = get_data_products_L2(self.L2)
        self.green_present = 'Green' in self.data_products
        self.red_present = 'Red' in self.data_products
        self.texp = self.header['ELAPSED']

        self.compute_statistics()
        
        
    def compute_statistics(self):
        """
        Compute various metrics of dispersion of the per-order BJD values
        """
        # compute weighted Barycentric RV correction
        x = self.df_RV['Bary_RVC']
        w = self.df_RV['CCF Weights']
        self.CCFBCV = np.sum(w * x) / np.sum(w)

        # compute weighted BJD (this should be computed elsewhere and read from the L2 header)
        x = self.df_RV['CCFBJD']
        w = self.df_RV['CCF Weights']
        self.CCFBJD = np.sum(w * x) / np.sum(w)

        # compute per-order BJD differences
        self.df_RV['Delta_CCFBJD'] = self.df_RV['CCFBJD'].copy()
        self.df_RV['Delta_CCFBJD'] -= self.CCFBJD
        #    compute weighted standard deviation
        x = self.df_RV['Delta_CCFBJD']
        w = self.df_RV['CCF Weights']
        nonzero_mask = w != 0
        wmean = np.sum(w * x) / np.sum(w)
        var_pop = np.sum(w * (x - wmean)**2) / np.sum(w) # weighted variance
        self.Delta_CCFBJD_weighted_std = np.sqrt(var_pop) * 24*60*60  # seconds
        self.Delta_CCFBJD_weighted_range = (x[nonzero_mask].max() - x[nonzero_mask].min()) * 24*60*60  # seconds

        # compute per-order Barycentric RV differences
        self.df_RV['Delta_Bary_RVC'] = self.df_RV['Bary_RVC'].copy()
        self.df_RV['Delta_Bary_RVC'] -= self.CCFBCV
        #    compute weighted standard deviation
        x = self.df_RV['Delta_Bary_RVC']
        wmean = np.sum(w * x) / np.sum(w)
        var_pop = np.sum(w * (x - wmean)**2) / np.sum(w) # weighted variance
        self.Delta_Bary_RVC_weighted_std = np.sqrt(var_pop) * 1000 # m/s
        self.Delta_Bary_RVC_weighted_range = (x[nonzero_mask].max() - x[nonzero_mask].min()) * 1000 # m/s


In [4]:
L2 = KPF2.from_fits("KP.20241022.39422.56_L2.fits")
myL2 = AnalyzeL2(L2)
print(myL2.df_RV)

    orderlet1  orderlet2  orderlet3  s_wavelength  e_wavelength  segment no.  \
0         0.0        0.0        0.0   4505.867807   4465.215086            0   
1         0.0        0.0        0.0   4538.949726   4497.995105            1   
2         0.0        0.0        0.0   4572.521116   4531.260119            2   
3         0.0        0.0        0.0   4606.592919   4565.020491            3   
4         0.0        0.0        0.0   4641.175961   4599.287622            4   
..        ...        ...        ...           ...           ...          ...   
62        0.0        0.0        0.0   8226.392913   8151.291145           27   
63        0.0        0.0        0.0   8337.435474   8261.310141           28   
64        0.0        0.0        0.0   8451.515446   8374.341090           29   
65        0.0        0.0        0.0   8568.759685   8490.509001           30   
66        0.0        0.0        0.0   8689.300786   8609.946569           31   

    order no.   RV  RV error    CAL RV 

In [5]:
print(myL2.CCFBCV)

0.05485045777642879


In [7]:
myL2.df_RV['Perc_Delta_Bary_RV'] = (myL2.df_RV['Delta_Bary_RVC'].copy() / myL2.CCFBCV) * 100
myL2.df_RV['Perc_Delta_Bary_RV'].max()

np.float64(0.9409079347137799)

In [8]:
nonzero_mask = myL2.df_RV['CCF Weights'] != 0
x = myL2.df_RV['Perc_Delta_Bary_RV']
myL2.Max_Perc_Delta_Bary_RV = x[nonzero_mask].max()
myL2.Min_Perc_Delta_Bary_RV = x[nonzero_mask].min()
print(f"max: {myL2.Max_Perc_Delta_Bary_RV}, min: {myL2.Min_Perc_Delta_Bary_RV}")

max: 0.6894375932041458, min: -0.47031634755679774
